In [9]:
import os
import shutil
import re
from sklearn.model_selection import train_test_split

# Paths (update these)
image_dir = "D:/Shaily_DeepLearning/DeepL/New_Detection2/images/train"
label_dir = "D:/Shaily_DeepLearning/DeepL/New_Detection2/labels/train"
output_dir = "D:/Shaily_DeepLearning/DeepL/New_Detection2/"

# Create output directories (if they don't exist)
os.makedirs(f"{output_dir}/images/val", exist_ok=True)
os.makedirs(f"{output_dir}/labels/val", exist_ok=True)

# Regex patterns for classification
crack_pattern = re.compile(r".*_C\d+.*\.(?:jpg|jpeg|png|bmp|gif|webp)$", re.IGNORECASE)
delam_pattern = re.compile(r".*_D\d+.*\.(?:jpg|jpeg|png|bmp|gif|webp)$", re.IGNORECASE)

# Classify files using regex patterns
crack_files = []
delam_files = []

for img_file in os.listdir(image_dir):
    if crack_pattern.match(img_file):
        crack_files.append(img_file)
    elif delam_pattern.match(img_file):
        delam_files.append(img_file)

print(f"Found {len(crack_files)} crack images")  # Debug output
print(f"Found {len(delam_files)} delam images")  # Debug output

# Split each class separately (stratified split)
train_crack, val_crack = train_test_split(crack_files, test_size=0.2, random_state=42)
train_delam, val_delam = train_test_split(delam_files, test_size=0.2, random_state=42)

# Function to move files to val directories
def move_to_val(files):
    for img_file in files:
        # Move image
        src_img = os.path.join(image_dir, img_file)
        dst_img = os.path.join(output_dir, "images/val", img_file)
        shutil.move(src_img, dst_img)
        
        # Move corresponding label
        label_file = os.path.splitext(img_file)[0] + '.txt'
        src_label = os.path.join(label_dir, label_file)
        dst_label = os.path.join(output_dir, "labels/val", label_file)
        shutil.move(src_label, dst_label)

# Move validation files
move_to_val(val_crack)
move_to_val(val_delam)

# Print results
print("\nFinal Counts:")
print(f"Total Crack Images: {len(crack_files)}")
print(f"Total Delam Images: {len(delam_files)}")
print(f"Train Crack: {len(train_crack)} | Val Crack: {len(val_crack)}")
print(f"Train Delam: {len(train_delam)} | Val Delam: {len(val_delam)}")
print("Validation files moved successfully!")

Found 340 crack images
Found 148 delam images

Final Counts:
Total Crack Images: 340
Total Delam Images: 148
Train Crack: 272 | Val Crack: 68
Train Delam: 118 | Val Delam: 30
Validation files moved successfully!


In [17]:
import os
import cv2

# Paths
image_dir = r"D:\ShailyDL\DeepL\FINAL_DS_Split\images\train"
label_dir = r"D:\ShailyDL\DeepL\FINAL_DS_Split\labels\train"
output_dir = r"D:\ShailyDL\DeepL\FINAL_DS_Split\ann_images\train"

# Make output dir if not exists
os.makedirs(output_dir, exist_ok=True)

# Class names (based on your YAML)
class_names = ['Delamination', 'Crack', 'NoDefect']

# Color mapping corrected to match class indices
class_colors = {
    0: (98, 0, 255),     # Delamination
    1: (0, 238, 196),    # Crack
    2: (255, 255, 0)     # NoDefect
}

# Iterate over images
for image_name in os.listdir(image_dir):
    if not image_name.lower().endswith(('.jpg', '.jpeg', '.png', '.bmp', '.webp')):
        continue

    image_path = os.path.join(image_dir, image_name)
    img = cv2.imread(image_path)
    if img is None:
        print(f"⚠️ Couldn't read image: {image_name}")
        continue

    h, w = img.shape[:2]

    label_name = os.path.splitext(image_name)[0] + '.txt'
    label_path = os.path.join(label_dir, label_name)

    if not os.path.exists(label_path):
        print(f"❌ Missing label for: {image_name}")
        continue

    with open(label_path, 'r') as f:
        for line in f.readlines():
            parts = line.strip().split()
            if len(parts) != 5:
                print(f"⚠️ Invalid label format in {label_name}")
                continue

            cls_id, xc, yc, bw, bh = map(float, parts)
            cls_id = int(cls_id)

            x1 = int((xc - bw / 2) * w)
            y1 = int((yc - bh / 2) * h)
            x2 = int((xc + bw / 2) * w)
            y2 = int((yc + bh / 2) * h)

            # Draw bounding box
            color = class_colors.get(cls_id, (255, 255, 255))
            cv2.rectangle(img, (x1, y1), (x2, y2), color, 2)

            # Add class label at center of bbox
            label_text = class_names[cls_id] if cls_id < len(class_names) else f'Class {cls_id}'
            center_x = int((x1 + x2) / 2)
            center_y = int((y1 + y2) / 2)

            font = cv2.FONT_HERSHEY_SIMPLEX
            font_scale = 0.9  # Bigger font
            thickness = 2

            # Get text size for center alignment
            text_size, _ = cv2.getTextSize(label_text, font, font_scale, thickness)
            text_w, text_h = text_size
            text_x = center_x - text_w // 2
            text_y = center_y + text_h // 2

            cv2.putText(img, label_text, (text_x, text_y), font, font_scale, color, thickness, lineType=cv2.LINE_AA)

    out_path = os.path.join(output_dir, image_name)
    cv2.imwrite(out_path, img)

print("\n✅ All images annotated and saved!")
print(f"Check folder: {output_dir}")



✅ All images annotated and saved!
Check folder: D:\ShailyDL\DeepL\FINAL_DS_Split\ann_images\train


In [15]:
import os
import cv2

# Input image and label paths
image_path = r"D:\ShailyDL\DeepL\FINAL_DS_Split\images\train\C185_aug.jpg"
label_path = r"D:\ShailyDL\DeepL\FINAL_DS_Split\labels\train\C185_aug.txt"

# Output directory
output_dir = r"D:\ShailyDL\DeepL\FINAL_DS_Split\ann_images\train"
os.makedirs(output_dir, exist_ok=True)

# Class names and color map
class_names = ['Delamination', 'Crack', 'NoDefect']
class_colors = {
    0: (98, 0, 255),
    1: (0, 238, 196),
    2: (255, 255, 0)
}

# Load image
img = cv2.imread(image_path)
if img is None:
    print(f"⚠️ Couldn't read image: {image_path}")
    exit()

h, w = img.shape[:2]

# Check if label exists
if not os.path.exists(label_path):
    print(f"❌ Missing label file: {label_path}")
    exit()

# Draw annotations
with open(label_path, 'r') as f:
    for line in f.readlines():
        parts = line.strip().split()
        if len(parts) != 5:
            print(f"⚠️ Invalid label format in {label_path}")
            continue

        cls_id, xc, yc, bw, bh = map(float, parts)
        cls_id = int(cls_id)

        x1 = int((xc - bw / 2) * w)
        y1 = int((yc - bh / 2) * h)
        x2 = int((xc + bw / 2) * w)
        y2 = int((yc + bh / 2) * h)

        # Draw bounding box
        color = class_colors.get(cls_id, (255, 255, 255))
        cv2.rectangle(img, (x1, y1), (x2, y2), color, 2)

        # Draw class label at bbox center
        label_text = class_names[cls_id] if cls_id < len(class_names) else f'Class {cls_id}'
        center_x = int((x1 + x2) / 2)
        center_y = int((y1 + y2) / 2)

        font = cv2.FONT_HERSHEY_SIMPLEX
        font_scale = 0.9
        thickness = 2

        # Text size for centering
        text_size, _ = cv2.getTextSize(label_text, font, font_scale, thickness)
        text_w, text_h = text_size
        text_x = center_x - text_w // 2
        text_y = center_y + text_h // 2

        # Optional: black rectangle background for better visibility
        cv2.rectangle(img, (text_x - 4, text_y - text_h - 4), (text_x + text_w + 4, text_y + 4), (0, 0, 0), -1)

        # Draw label text
        cv2.putText(img, label_text, (text_x, text_y), font, font_scale, (255, 255, 255), thickness, lineType=cv2.LINE_AA)

# Save the annotated image
output_path = os.path.join(output_dir, os.path.basename(image_path))
cv2.imwrite(output_path, img)

print("\n✅ Annotation done for one image!")
print(f"📂 Saved to: {output_path}")



✅ Annotation done for one image!
📂 Saved to: D:\ShailyDL\DeepL\FINAL_DS_Split\ann_images\train\C185_aug.jpg
